In [ ]:
import os

GPU_ID = "3"
os.environ['CUDA_VISIBLE_DEVICES'] = GPU_ID

import sys
sys.path.append('/home/rustam/hyperspecter_segmentation/makitorch')
sys.path.append('/home/rustam/hyperspecter_segmentation/')

PREFIX_INFO_PATH = '/home/rustam/hyperspecter_segmentation/danil_cave/kfolds_data_with_other/kfold0/data'
PATH_DATA = '/raid/rustam/hyperspectral_dataset/new_cropped_hsi_data'
NUM_CLASSES = 8

import random
from multiprocessing.dummy import Pool
from multiprocessing import shared_memory
import copy
from makitorch import *
from makitorch.dataset_remapper import DatasetRemapper
import math
import numpy as np
import numba as nb
import comet_ml
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms as T
import torchvision.transforms.functional as TF
from torchvision import utils
import cv2
from Losses import FocalLoss
import matplotlib.pyplot as plt

import seaborn as sns
import json
from tqdm import tqdm

from sklearn.decomposition import PCA
from makitorch.architectures.U2Net import U2Net

from hsi_dataset_api import HsiDataset

from makitorch.dataloaders.HsiDataloader import HsiDataloader
from makitorch.architectures.Unet import Unet, UnetWithFeatureSelection
from makitorch.loss import muti_bce_loss_fusion
from sklearn.metrics import jaccard_score
np.set_printoptions(suppress=True)


from makitorch.data_tools.augmentation import DataAugmentator
from makitorch.data_tools.augmentation import BaseDataAugmentor
from makitorch.data_tools.preprocessing import BaseDataPreprocessor
from makitorch.data_tools.preprocessing import DataPreprocessor

from typing import Callable, Optional, Union

import torch
from sklearn.utils import shuffle
from hsi_dataset_api import HsiDataset




@nb.njit
def cut_into_parts(
        image: np.ndarray, mask: np.ndarray, h_parts: int, 
        w_parts: int, h_win: int, w_win: int):
    image_parts_list = []
    mask_parts_list = []

    for h_i in range(h_parts):
        for w_i in range(w_parts):
            img_part = image[:, 
                h_i * h_win: (h_i+1) * h_win, 
                w_i * w_win: (w_i+1) * w_win
            ]
            mask_part = mask[
                h_i * h_win: (h_i+1) * h_win, 
                w_i * w_win: (w_i+1) * w_win
            ]

            image_parts_list.append(img_part)
            mask_parts_list.append(mask_part)
    return image_parts_list, mask_parts_list


class ShmData:

    def __init__(self, shm_name, shape, dtype):
        self.shm_name = shm_name
        self.shape = shape
        self.dtype = dtype


class DatasetCreator:

    def __init__(
            self, 
            data_path: str,
            preprocessing: Optional[Union[DataPreprocessor, Callable]] = BaseDataPreprocessor(),
            indices = None,
            cut_window=(8, 8),
            map_mask_to_class=False,
            create_shared_memory=False,
            shuffle_then_prepared=False,
            path_old2new: str = None):
        self.dataset = HsiDataset(data_path)
        self.preprocessing = preprocessing
        self.cut_window = cut_window
        self.map_mask_to_class = map_mask_to_class
        self.create_shared_memory = create_shared_memory
        
        if path_old2new:
            self.old2new_mapper = DatasetRemapper(np.load(path_old2new))
        else:
            self.old2new_mapper = None

        self.images = []
        self.masks = []
        self._shm_imgs = None
        self._shm_masks = None

        for idx, data_point in tqdm(enumerate(self.dataset.data_iterator(opened=True, shuffle=False))):
            if indices is not None and idx not in indices:
                continue
            image, mask = data_point.hsi, data_point.mask

            if self.old2new_mapper:
                _, mask = self.old2new_mapper(None, mask)

            if cut_window is not None:
                image_parts, mask_parts = self._cut_with_window(image, mask, cut_window)
                self.images += image_parts
                self.masks += mask_parts
            else:
                self.images.append(image)
                self.masks.append(mask)
        print("Preprocess data...")
        if self.preprocessing is not None:
            self.images, self.masks = self.preprocessing(
                self.images, self.masks, map_mask_to_class=map_mask_to_class
            )

        if shuffle_then_prepared:
            self.images, self.masks = shuffle(self.images, self.masks)

        # Create shared memory
        if create_shared_memory:
            print('Create shared memory...')
            # First - map images and masks into np
            self.images = np.asarray(self.images, dtype=np.float32)
            self.masks = np.asarray(self.masks, dtype=np.int64)
            # Imgs
            shm_imgs = shared_memory.SharedMemory(create=True, size=self.images.nbytes)
            shm_imgs_arr = np.ndarray(self.images.shape, dtype=self.images.dtype, buffer=shm_imgs.buf)
            shm_imgs_arr[:] = self.images[:]
            self.images = shm_imgs_arr # Do not keep dublicate 
            self.data_shm_imgs = ShmData(
                shm_name=shm_imgs.name, shape=self.images.shape, 
                dtype=self.images.dtype
            )
            self._shm_imgs = shm_imgs
            # Masks
            shm_masks = shared_memory.SharedMemory(create=True, size=self.masks.nbytes)
            shm_masks_arr = np.ndarray(self.masks.shape, dtype=self.masks.dtype, buffer=shm_masks.buf)
            shm_masks_arr[:] = self.masks[:]
            self.masks = shm_masks_arr # Do not keep dublicate 
            self.data_shm_masks = ShmData(
                shm_name=shm_masks.name, shape=self.masks.shape,
                dtype=self.masks.dtype
            )
            self._shm_masks = shm_masks
            print("Shared memory are created for imgs and masks!")

    def close_shm(self):
        if self.create_shared_memory:
            # Make sure there is no reference data to images/masks
            del self.images
            del self.masks
            self.images = []
            self.masks = []
            # Close and unlink
            if self._shm_masks is not None:
                self._shm_masks.close()
                self._shm_masks.unlink()
                self._shm_masks = None

            if self._shm_imgs is not None:
                self._shm_imgs.close()
                self._shm_imgs.unlink()
                self._shm_imgs = None
            print("Shared memory for masks and images are success cleared!")
                    
    
    def _cut_with_window(self, image, mask, cut_window):
        assert len(cut_window) == 2
        h_win, w_win = cut_window
        _, h, w = image.shape
        h_parts = h // h_win
        w_parts = w // w_win
        if h % h_win != 0:
            print(f"{h % h_win} pixels will be dropped by h axis. Input shape={image.shape}")

        if w % w_win != 0:
            print(f"{w % w_win} pixels will be dropped by w axis. Input shape={image.shape}")
        return cut_into_parts(
            image=image, mask=mask, h_parts=h_parts, w_parts=w_parts,
            h_win=h_win, w_win=w_win
        )


# Define a `worker_init_fn` that configures each dataset copy differently
def worker_init_fn(worker_id):
    worker_info = torch.utils.data.get_worker_info()
    dataset = worker_info.dataset
    shared_memory_imgs_data: ShmData = dataset.shared_memory_imgs_data
    shared_memory_masks_data: ShmData = dataset.shared_memory_masks_data
    if shared_memory_imgs_data is not None and shared_memory_masks_data is not None:
        # Take array from memory
        existing_shm_imgs = shared_memory.SharedMemory(name=shared_memory_imgs_data.shm_name)
        dataset_imgs_np = np.ndarray(
            shared_memory_imgs_data.shape, 
            dtype=shared_memory_imgs_data.dtype, buffer=existing_shm_imgs.buf
        )
        dataset.shm_imgs = existing_shm_imgs
        existing_shm_masks = shared_memory.SharedMemory(name=shared_memory_masks_data.shm_name)
        dataset_masks_np = np.ndarray(
            shared_memory_masks_data.shape, 
            dtype=shared_memory_masks_data.dtype, buffer=existing_shm_masks.buf
        )
        dataset.shm_masks = existing_shm_masks
    else:
        assert dataset.images is not None and dataset.masks is not None
        dataset_imgs_np = dataset.images
        dataset_masks_np = dataset.masks
    overall_start = 0
    overall_end = len(dataset_imgs_np)
    # configure the dataset to only process the split workload
    per_worker = int(math.ceil((overall_end - overall_start) / float(worker_info.num_workers)))
    worker_id = worker_info.id
    start = overall_start + worker_id * per_worker
    end = min(start + per_worker, overall_end)
    dataset.images = list(dataset_imgs_np[start:end])
    dataset.masks = list(dataset_masks_np[start:end])


class HsiDataloaderCutter(torch.utils.data.IterableDataset):
    def __init__(
            self, 
            images, masks,
            augmentation: Optional[Union[DataAugmentator, Callable]] = BaseDataAugmentor(),
            shuffle_data=False,
            cut_window=(8, 8),
            map_mask_to_class=False,
            shared_memory_imgs_data: ShmData = None,
            shared_memory_masks_data: ShmData = None
        ):
        super().__init__()
        self.shuffle_data = shuffle_data
        self.augmentation = augmentation
        if cut_window is not None and cut_window[0] == 1 and cut_window[1] == 1:
            self.ignore_image_augs = True
        else:
            self.ignore_image_augs = False
        self.cut_window = cut_window
        self.map_mask_to_class = map_mask_to_class
        self.shared_memory_imgs_data = shared_memory_imgs_data
        self.shared_memory_masks_data = shared_memory_masks_data
        
        self.shm_imgs: shared_memory.SharedMemory = None
        self.shm_masks: shared_memory.SharedMemory = None

        self.images = images
        self.masks = masks

    def __iter__(self):
        assert self.images is not None and self.masks is not None
        if self.shuffle_data:
            self.images, self.masks = shuffle(self.images, self.masks)
        
        for image, mask in zip(self.images, self.masks):
            yield self.augmentation(
                image, mask, 
                map_mask_to_class=self.map_mask_to_class,
                ignore_image_augs=self.ignore_image_augs
            )

            

device = 'cuda:0'
pca_explained_variance = np.load(f'{PREFIX_INFO_PATH}/kfold0_PcaExplainedVariance_.npy')
pca_mean = np.load(f'{PREFIX_INFO_PATH}/kfold0_PcaMean.npy')
pca_components = np.load(f'{PREFIX_INFO_PATH}/kfold0_PcaComponents.npy')


def pca_transformation(x):
    if len(x.shape) == 3:
        x_t = x.reshape((x.shape[0], -1)) # (C, H, W) -> (C, H * W)
        x_t = np.transpose(x_t, (1, 0)) # (C, H * W) -> (H * W, C)
        x_t = x_t - pca_mean
        x_t = np.dot(x_t, pca_components.T) / np.sqrt(pca_explained_variance)
        return x_t.reshape((x.shape[1], x.shape[2], pca_components.shape[0])).astype(np.float32, copy=False) # (H, W, N)
    elif len(x.shape) == 4:
        # x - (N, C, H, W)
        x_t = np.transpose(x, (0, 2, 3, 1)) # (N, C, H, W) -> (N, H, W, C)
        x_t = x_t - pca_mean
        x_t = np.dot(x_t, pca_components.T) / np.sqrt(pca_explained_variance)
        x_t = np.transpose(x_t, (0, -1, 1, 2)) # (N, H, W, C) -> (N, C, H, W)
        return x_t.astype(np.float32, copy=False)
    else:
        raise ValueError(f"Unknown shape={x.shape}, must be of len 3 or 4.")

def standartization(img, mean, std):
    img -= mean
    img /= std
    return img

def standartization_pool(mean, std):
    # X shape - (N, C, H, W)
    # from shape (comp,) -> (1, comp, 1, 1)
    mean = np.expand_dims(np.expand_dims(np.array(mean, dtype=np.float32), axis=-1), axis=-1)
    std = np.expand_dims(np.expand_dims(np.array(std, dtype=np.float32), axis=-1), axis=-1)
    
    return lambda x: standartization(x, mean=mean, std=std)


def mask2class(mask):
    # Calculate which class have more pixel count
    max_value = -1
    pixel_count = -1
    for class_indx in np.unique(mask):
        pix_count_s = np.sum(mask == class_indx)
        if pix_count_s > pixel_count:
            max_value = class_indx
            pixel_count = pix_count_s
    assert max_value != -1
    return np.array([max_value], dtype=np.int64) 


def preprocessing(imgs, masks, map_mask_to_class=False, split_size=256):
    with open(f'{PREFIX_INFO_PATH}/data_standartization_params_kfold0.json', 'r') as f:
        data_standartization_params = json.load(f)
    mean = data_standartization_params.get('means')
    std = data_standartization_params.get('stds')
    assert mean is not None and std is not None
    print('Create np array of imgs and masks...')
    imgs_np = np.asarray(imgs, dtype=np.float32) # (N, 237, 1, 1)
    masks_np = np.asarray(masks, dtype=np.int64) # (N, 1, 1, 3)
    print("Split imgs dataset...")
    imgs_split_np = np.array_split(imgs_np, split_size) # (split_size, Ns, 237, 1, 1)
    print('Start preprocess images...')
    # Wo PCA
    # _images = [np.transpose(image, (1, 2, 0)) for image in imgs]
    # W Pca
    with Pool(18) as p:
        _images = list(tqdm(p.imap(
                pca_transformation, 
                imgs_split_np,
                #chunksize=1
            ), total=len(imgs_split_np))
        )
        _images = list(tqdm(p.imap(
            standartization_pool(mean=mean, std=std), 
            _images,
            #chunksize=1
            ), total=len(imgs_split_np))
        )
    _images = list(np.concatenate(_images, axis=0)) # (split_size, Ns, 237, 1, 1) -> (split_size * Ns, 237, 1, 1)
    print("Preprocess masks...")
    _masks = list(np.transpose(masks_np[..., 0:1], (0, -1, 1, 2)))
    print("Finish preprocess!")
    if map_mask_to_class:
        _masks = [mask2class(mask) for mask in _masks]
    return _images, _masks


def test_augmentation(image, mask, **kwargs):
    image = torch.from_numpy(image)
    #image = (image - image.min()) / (image.max() - image.min())
    
    mask = torch.from_numpy(mask)
    mask = torch.squeeze(mask, 0)
    return image, mask



def aug_random_rotate(image, mask, map_mask_to_class=False, **kwargs):
    angle = T.RandomRotation.get_params((-30, 30))
    image = TF.rotate(image, angle, interpolation=T.InterpolationMode.BILINEAR)
    if not map_mask_to_class:
        mask = TF.rotate(mask, angle, interpolation=T.InterpolationMode.NEAREST)
    return image, mask


def aug_flip_horizontal(image, mask, map_mask_to_class=False, **kwargs):
    if torch.rand(1) > 0.5:
        image = TF.hflip(image)
        if not map_mask_to_class:
            mask = TF.hflip(mask)
    return image, mask


def aug_flip_vertical(image, mask, map_mask_to_class=False, **kwargs):
    if torch.rand(1) > 0.5:
        image = TF.vflip(image)
        if not map_mask_to_class:
            mask = TF.vflip(mask)
    return image, mask


MASK_AUG_SCALE = 100
MASK_AUG_COMPARE = 90
RandomEraseTorch = T.RandomErasing(
    p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), 
    value='random', inplace=False
)
def aug_random_erase(image, mask, map_mask_to_class=False, **kwargs):
    # Create mask in order to take area of aug
    mask_aug_area = torch.ones(
        1, image.shape[1], image.shape[2], 
        dtype=image.dtype, device=image.device
    ) * MASK_AUG_SCALE
    in_x = torch.cat([image, mask_aug_area], dim=0)
    # Apply aug
    img_aug = RandomEraseTorch(in_x)
    image = img_aug[:-1]
    if not map_mask_to_class:
        mask_aug = img_aug[-1:]
        # Take mask and reverse values
        # 0 - cutout zone, 1 - good zone
        mask_aug = (mask_aug > MASK_AUG_COMPARE).long()
        mask = mask * mask_aug
    return image, mask


AUGS_LIST = [
    aug_random_rotate,
    aug_flip_horizontal,
    aug_flip_vertical,
    aug_random_erase
]


def augmentation(image, mask, map_mask_to_class=False, ignore_image_augs=False):
    image = torch.from_numpy(image)
    mask = torch.from_numpy(mask)
    if not ignore_image_augs:
        for aug_func in AUGS_LIST:
            image, mask = aug_func(image, mask, map_mask_to_class=map_mask_to_class)
    
    #image = (image - image.min()) / (image.max() - image.min())
    mask = torch.squeeze(mask, 0)
    return image, mask


USE_SHM = True

if USE_SHM:
    NUM_WORKERS = 10
else:
    NUM_WORKERS = 5

N_REPEAT_EXP = 2

T_0_param = 2
T_mult_param = 1

In [ ]:
cut_window = (128, 128)
batch_size = 1
test_indices = np.load(f'{PREFIX_INFO_PATH}/kfold0_indx_unknown_test.npy')
val_indices = np.load(f'{PREFIX_INFO_PATH}/kfold0_indx_test.npy')
train_indices = np.load(f'{PREFIX_INFO_PATH}/kfold0_indx_train.npy')

# Load and preprocess data
# Train
if USE_SHM:
    print("Using shared memory...")
    dataset_creator_train = DatasetCreator(
        PATH_DATA, preprocessing=preprocessing, 
        indices=train_indices, cut_window=cut_window,
        create_shared_memory=True, shuffle_then_prepared=True,
        path_old2new=f'{PREFIX_INFO_PATH}/index2class.npy'
    )
    dataset_train = HsiDataloaderCutter(
        images=None, masks=None,
        augmentation=augmentation,
        shuffle_data=True, cut_window=cut_window,
        shared_memory_imgs_data=dataset_creator_train.data_shm_imgs,
        shared_memory_masks_data=dataset_creator_train.data_shm_masks,
    )
else:
    dataset_creator_train = DatasetCreator(
        PATH_DATA, preprocessing=preprocessing, 
        indices=train_indices, cut_window=cut_window,
        create_shared_memory=False, shuffle_then_prepared=True,
        path_old2new=f'{PREFIX_INFO_PATH}/index2class.npy'
    )
    dataset_train = HsiDataloaderCutter(
        images=dataset_creator_train.images, masks=dataset_creator_train.masks,
        augmentation=augmentation,
        shuffle_data=True, cut_window=cut_window,
    )
print(f"Number of workers={NUM_WORKERS}")
train_loader = torch.utils.data.DataLoader(
    dataset_train, batch_size=batch_size, 
    num_workers=NUM_WORKERS, pin_memory=False, prefetch_factor=2,
    worker_init_fn=worker_init_fn, drop_last=True
)
# Validation
dataset_creator_val = DatasetCreator(
    PATH_DATA, preprocessing=preprocessing, 
    indices=val_indices, cut_window=None,
    create_shared_memory=False,
    path_old2new=f'{PREFIX_INFO_PATH}/index2class.npy'
)
dataset_val = HsiDataloaderCutter(
    images=dataset_creator_val.images, masks=dataset_creator_val.masks,
    augmentation=test_augmentation,
    shuffle_data=False, cut_window=None,
)
val_loader = torch.utils.data.DataLoader(dataset_val, batch_size=1)

# Test
dataset_creator_test = DatasetCreator(
    PATH_DATA, preprocessing=preprocessing, 
    indices=test_indices, cut_window=None,
    create_shared_memory=False,
    path_old2new=f'{PREFIX_INFO_PATH}/index2class.npy'
)
dataset_test = HsiDataloaderCutter(
    images=dataset_creator_test.images, masks=dataset_creator_test.masks,
    augmentation=test_augmentation,
    shuffle_data=False, cut_window=None,
)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=1)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
part_index = test_indices

In [ ]:
part_index = test_indices

In [ ]:
# Test
dataset_creator_test = DatasetCreator(
    PATH_DATA, preprocessing=preprocessing, 
    indices=part_index, cut_window=None,
    create_shared_memory=False,
    path_old2new=f'{PREFIX_INFO_PATH}/index2class.npy'
)
dataset_test = HsiDataloaderCutter(
    images=dataset_creator_test.images, masks=dataset_creator_test.masks,
    augmentation=test_augmentation,
    shuffle_data=False, cut_window=None,
)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=1)

In [ ]:
test_iter = iter(test_loader)

In [ ]:
test_image, test_mask = next(test_iter)

In [ ]:
test_mask.shape

In [ ]:
sns.heatmap(test_mask[0])

In [ ]:
sns.heatmap(test_image[0, 0])

In [ ]:
dataset_creator_test = DatasetCreator(
    PATH_DATA, preprocessing=preprocessing, 
    indices=np.asarray(range(362)), cut_window=None,
    create_shared_memory=False,
    path_old2new=f'{PREFIX_INFO_PATH}/index2class.npy'
)

In [ ]:
images=dataset_creator_test.images; masks=dataset_creator_test.masks
N = len(images)
counter = 0
size = 5
part_s = (N // (size * size)) + (N % (size * size) != 0)
for p_i in range(part_s):
    fig = plt.figure(figsize=(12, 12))
    for i in range(1, min(size * size + 1, N - p_i * size * size)):
        img = masks[counter][0]
        fig.add_subplot(size, size, i)
        plt.axis('off')
        plt.title(f"{counter}")
        sns.heatmap(img, vmin=0, vmax=8)
        counter += 1
    plt.savefig(f"res_{p_i}", dpi=350)
    plt.close(fig)